In [14]:
from sqlalchemy import create_engine
import pandas as pd

# Define your database credentials
USERNAME = 'admin'
PASSWORD = 'isba_4715'
HOST = 'isba-dev-01.ctqaesoyaych.us-east-1.rds.amazonaws.com'  # Example: mydatabase.123456789012.us-west-1.rds.amazonaws.com
PORT = '3306'
DATABASE = 'SQL_Project_Spring_2024'

# Create the database URL for SQLAlchemy
DATABASE_URL = f"mysql+pymysql://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}"

# Create a database engine
engine = create_engine(DATABASE_URL)

# Descriptive Analytics Query: Average and Maximum Premiums by Metal Level in Each County
descriptive_query = """
WITH PremiumStats AS (
    SELECT 
        c.name AS county_name,
        p.metal_level,
        AVG(p.premium) AS avg_premium,
        MIN(p.premium) AS min_premium,
        MAX(p.premium) AS max_premium
    FROM Counties c
    JOIN ZipCodes z ON c.county_id = z.county_id
    JOIN PlanZipCodes pz ON z.zip_id = pz.zip_id
    JOIN Plans p ON pz.db_id = p.db_id
    GROUP BY c.name, p.metal_level
)
SELECT 
    county_name,
    metal_level,
    avg_premium,
    min_premium,
    max_premium
FROM PremiumStats
ORDER BY county_name, metal_level;
"""

# Diagnostic Analytics Query: Analyze Deductible Trends by Plan Type
diagnostic_query = """
WITH DeductibleCorrelation AS (
    SELECT 
        c.name AS county_name,
        p.metal_level,
        AVG(p.premium) AS avg_premium,
        AVG(d.amount) AS avg_deductible,
        COUNT(d.db_id) AS count_deductibles
    FROM Counties c
    JOIN ZipCodes z ON c.county_id = z.county_id
    JOIN PlanZipCodes pz ON z.zip_id = pz.zip_id
    JOIN Plans p ON pz.db_id = p.db_id
    JOIN Deductibles d ON p.db_id = d.db_id
    GROUP BY c.name, p.metal_level
)
SELECT 
    county_name,
    metal_level,
    avg_premium,
    avg_deductible,
    count_deductibles
FROM DeductibleCorrelation
ORDER BY county_name, metal_level;
"""

# Execute and display results for Descriptive Analytics
descriptive_results = pd.read_sql(descriptive_query, con=engine)
print("Descriptive Analytics Results:")
print(descriptive_results)

# Execute and display results for Diagnostic Analytics
diagnostic_results = pd.read_sql(diagnostic_query, con=engine)
print("\nDiagnostic Analytics Results:")
print(diagnostic_results)

# Close the connection when done
engine.dispose()


Descriptive Analytics Results:
   county_name   metal_level  avg_premium  min_premium  max_premium
0       Apache  Catastrophic       151.23       151.23       151.23
1      Cochise  Catastrophic       184.36       184.36       184.36
2     Coconino  Catastrophic       151.23       151.23       151.23
3         Gila  Catastrophic       111.20       111.20       111.20
4       Graham  Catastrophic       184.36       184.36       184.36
5     Greenlee  Catastrophic       184.36       184.36       184.36
6       La Paz  Catastrophic       163.33       163.33       163.33
7     Maricopa  Catastrophic        65.72        65.72        65.72
8       Mohave  Catastrophic       151.23       151.23       151.23
9       Navajo  Catastrophic       151.23       151.23       151.23
10        Pima  Catastrophic       119.11       119.11       119.11
11       Pinal  Catastrophic       179.75       179.75       179.75
12  Santa Cruz  Catastrophic       131.27       131.27       131.27
13     Yavapai  C

In [ ]:
print